In [1]:
%matplotlib inline

import sys
import os
import pathlib
sys.path.append('/g/data/mn51/users/mtb563/toolbox/modules')
import utils

# User defined
indices = ['TXm','TXx','TXge35','TXge40','TXge45','TNle02','EHF']
bc_methods = ['raw','ACS-QME']
pathways = ['ssp370']

#Data specific information
training_data = 'AGCD-1960-2022'
project = 'CORDEX-CMIP6'
grid='AGCD-05i'
data_status = 'test-data'
data_version = 'v1-r1'

In [2]:
def get_dirpath_gcms():
    """returns the directory path within which the data is seperated per GCM"""
    return f'/g/data/ia39/australian-climate-service/{data_status}/{project}/bias-adjustment-{bc_status}/{grid}/{institution}/'

def get_fpath_infile(var):
    """returns the full path for the intended input file(s) for the specified variable"""
    ddir_p1 = get_dirpath_gcms()
    ddir_p2 = f'{gcm}/'+'{pathway}'+f'/{member}/{rcm[institution]}/{data_version}{"-"+bc_method+"-"+bc_training_data if bc_method != "raw" else ""}/day/VAR'+f'{"Adjust" if bc_method != "raw" else ""}/'
    fname = 'VAR'+f'{"Adjust" if bc_method != "raw" else ""}_{grid}_{gcm}_'+'{pathway}'+f'_{member}_{institution}_{rcm[institution]}_{data_version}{"-"+bc_method+"-"+bc_training_data if bc_method != "raw" else ""}_day_'+'{Y}01'+f'{"" if (bc_method == "raw" and institution == "BOM") else "01"}'+'-{Y}12'+f'{"" if (bc_method == "raw" and institution == "BOM") else "31"}.nc'
    infile = ddir_p1 + ddir_p2 + fname
    return infile.replace('VAR',var)

def get_dirpath_ofile():
    odir = f'/g/data/mn51/projects/work_package_4/climate_hazard_indices/heat/{data_status}/{project}/{grid}/{gcm}/{pathway}/{member}/{rcm[institution]}/{bc_method}/{bc_training_data}/day/{index}/'
    return odir

In [ ]:
#assert bc_method in ['ACS-QME','raw']
#assert bc_training_data in ['AGCD-1960-2022','BARRA-R2-1979-2022']

rcm = {'BOM':'BARPA-R','CSIRO':'CCAM-v2203-SN'}

for index in indices:
    for bc_method in bc_methods:
        if bc_method == 'raw':
            bc_status = 'input'
            bc_training_data = 'raw'
        else:
            bc_status = 'output'
            bc_training_data = training_data
        
        for institution in ['BOM','CSIRO']:
            gcms = os.listdir(get_dirpath_gcms())
            if 'ERA5' in gcms: gcms.remove('ERA5')
            for gcm in gcms:
                for pathway in pathways:
                    for member in os.listdir(f'{get_dirpath_gcms()}/{gcm}/{pathway}'):
                        pathlib.Path(get_dirpath_ofile()).mkdir(parents=True, exist_ok=True)
                        os.chdir(get_dirpath_ofile())

                        print(utils.display.txtcolor.BOLD + utils.display.txtcolor.BLUE + f'{index} {gcm} {rcm[institution]} {pathway} {member} {bc_method}' + utils.display.txtcolor.END)
                        
                        if index == 'EHF':
                            run_command =  (
                                f"python /g/data/mn51/users/mtb563/toolbox/data_processing/heat/calc_EHF.py"
                                f" --tasmax_fpath {get_fpath_infile('tasmax')}" 
                                f" --tasmax_varname tasmax{'Adjust' if bc_method != 'raw' else ''}"
                                f" --tasmin_fpath {get_fpath_infile('tasmin')}"
                                f" --tasmin_varname tasmin{'Adjust' if bc_method != 'raw' else ''}"
                                f" --driving_model {gcm}"
                                f" --downscaling_model {rcm[institution]}"
                                f" --bias_correction_method {bc_method} "
                                f" --pathway {pathway}"
                                f" --BPStartYr 1985"
                                f" --BPEndYr 2014" 
                                f" --StartYr 1980"
                                f" --EndYr 2099"
                                f" --annual_groupby Jul2Jun"
                                f" --ofile_drs VAR_{grid}_{gcm}_{pathway}_{member}_{institution}_{rcm[institution]}_{data_version}{'-'+bc_method+'-'+bc_training_data if bc_method != 'raw' else ''}_day_TPERIOD.nc"
                                f" --tidy_wkdir True"
                            )
                        
                        if index in ['TXm','TXx','TXge35','TXge40','TXge45','TXge50','TX90P','TNle02']:
                            #os.system(f'rm -rf {get_dirpath_ofile()}')
                            run_command =  (
                                f"python /g/data/mn51/users/mtb563/toolbox/data_processing/heat/calc_tas_indices.py"
                                f" --index {index}"
                                f" --tasmax_fpath {get_fpath_infile('tasmax')}" 
                                f" --tasmax_varname tasmax{'Adjust' if bc_method != 'raw' else ''}"
                                f" --tasmin_fpath {get_fpath_infile('tasmin')}"
                                f" --tasmin_varname tasmin{'Adjust' if bc_method != 'raw' else ''}"
                                f" --driving_model {gcm}"
                                f" --downscaling_model {rcm[institution]}"
                                f" --bias_correction_method {bc_method} "
                                f" --pathway {pathway}"
                                f" --BPStartYr 1985"
                                f" --BPEndYr 2014" 
                                f" --StartYr 1980"
                                f" --EndYr 2099"
                                f" --ofile_drs INDEX_{grid}_{gcm}_{pathway}_{member}_{institution}_{rcm[institution]}_{data_version}{'-'+bc_method+'-'+bc_training_data if bc_method != 'raw' else ''}_day_TPERIOD.nc"
                                f" --tidy_wkdir True"
                            )

                        os.system(run_command)
                        

TXm MPI-ESM1-2-HR BARPA-R ssp370 r1i1p1f1 raw
[########################################] | 100% Completed | 22.34 s
[########################################] | 100% Completed | 22.34 s
[                                        ] | 0% Completed | 15.45 sms


Traceback (most recent call last):
  File "/g/data/mn51/users/mtb563/toolbox/data_processing/heat/calc_tas_indices.py", line 196, in <module>
    main(args)
  File "/g/data/mn51/users/mtb563/toolbox/data_processing/heat/calc_tas_indices.py", line 147, in main
    utils.generalio.save_data(index,get_fname(index=inargs.index,tperiod=f'{Y}0101-{Y}{inargs.yearend.replace("-","")}').replace('day','annual'))
  File "/g/data/mn51/users/mtb563/toolbox/modules/generalio.py", line 145, in save_data
    ds.to_netcdf(ofile)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3/lib/python3.9/site-packages/xarray/core/dataarray.py", line 3959, in to_netcdf
    return to_netcdf(  # type: ignore  # mypy cannot resolve the overloads:(
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3/lib/python3.9/site-packages/xarray/backends/api.py", line 1225, in to_netcdf
    writes = writer.sync(compute=compute)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3/lib/python3.9/site-packages/xarra